In [ ]:
!pip install --upgrade scikit-learn
!pip install umap-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 58.3 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82807 sha256=06ddb554dc3c74bf799ab2b57db70041be6ed0d7dae54a6b48fe0e06970be0c9
  Stored in directory: /root/.cache/pip/wheels/a0/e8/c6/a37ea663620bd5200ea1ba0907ab3c217042c1d035ef606acc
  Created wheel for pynndescent: filename=pynndescent-0.5.10-py3-none-any.whl size=55615 sha256=e78d02810ecb2cb07255607116e64aa498bbf2f0917c4fcfa42b723fb3778183
  Stored in directory: /root/.cache/pip/wheels/4a/38/5d/f60a40a66a9512b7e5e8

In [ ]:
from sklearn.cluster import DBSCAN, OPTICS,HDBSCAN
import numpy as np
import pandas as pd
from google.colab import drive
import umap
import matplotlib.pyplot as plt
from sklearn.metrics import davies_bouldin_score, calinski_harabasz_score, silhouette_score


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

#input_path="/content/drive/MyDrive/TFM/Instagram.Post.json"

EMB_PATHS = '/content/drive/MyDrive/TFM/Embeddings/ImgEmbeddings/ImgEmbeddings.csv'

Mounted at /content/drive


In [17]:
ids = []
features = []
with open(EMB_PATHS, 'r') as csv_file:
    for line in csv_file:
      ids.append(line.split(',')[0])
      emb = [float(value) for value in line.split(',')[1:]]
      sublists = [emb[i:i+768] for i in range(0, len(emb), 768)]
      mean_embedding = np.mean(sublists, axis=0)
      #aux_emb = np.dot(sublists)
      features.append(mean_embedding)



In [18]:
def generate_clusters(message_embeddings,
                    n_neighbors,
                    n_components,
                    min_cluster_size,
                    random_state = None):
  """
  Genersate HDBSCAN cluster object after reducing embedding dimensionality with UMAP
  """

  umap_embeddings = (umap.UMAP(n_neighbors=n_neighbors,
                              n_components=n_components,
                              metric='cosine',
                              random_state=random_state)
                          .fit_transform(message_embeddings))

  clusters = DBSCAN(min_samples = min_cluster_size,
                              metric='euclidean').fit_predict(umap_embeddings)

  return clusters

In [22]:
def evaluate_clustering(title, features, labels):
    """
    Calculate clustering evaluation metrics for a given clustering result and print the results.

    Parameters:
    - title: A string, title for the clustering result.
    - features: Features used for clustering (numpy array or DataFrame).
    - labels: Cluster labels assigned by the clustering algorithm.
    """
    try:
      davies_bouldin = davies_bouldin_score(features, labels)
    except:
      davies_bouldin = 'Not applicable'

    try:
      calinski_harabasz = calinski_harabasz_score(features, labels)
    except:
      calinski_harabasz = 'Not applicable'

    try:
      silhouette = silhouette_score(features, labels)
    except:
      silhouette = 'Not applicable'
    try:
      n_clusters = len(set(labels))
    except:
      n_clusters = 'Not applicable'

    print(f"{title} Clustering Evaluation:")
    print(f"Davies-Bouldin Score: {davies_bouldin}")
    print(f"Calinski-Harabasz Score: {calinski_harabasz}")
    print(f"Silhouette Score: {silhouette}\n")
    print(f"Number of clusters: {n_clusters}\n")

In [23]:
for n_neighbors in (5, 50, 100):
  for n_components in (3, 5, 20 , 50, len(features[0])):
    for min_cluster_size in (50, 100, 500, 1000):
      cluster_labels = generate_clusters(features,
          n_neighbors, n_components, min_cluster_size)
      evaluate_clustering(f"DBSCAN-{n_neighbors}_{n_components}_{min_cluster_size}", features, cluster_labels)


DBSCAN-5_3_50 Clustering Evaluation:
Davies-Bouldin Score: 3.75210133043425
Calinski-Harabasz Score: 66.0826980631113
Silhouette Score: -0.021122138822611103

Number of clusters: 22

DBSCAN-5_3_100 Clustering Evaluation:
Davies-Bouldin Score: 3.083009827083056
Calinski-Harabasz Score: 66.30803546605537
Silhouette Score: -0.055510630634676136

Number of clusters: 5

DBSCAN-5_3_500 Clustering Evaluation:
Davies-Bouldin Score: Not applicable
Calinski-Harabasz Score: Not applicable
Silhouette Score: Not applicable

Number of clusters: 1

DBSCAN-5_3_1000 Clustering Evaluation:
Davies-Bouldin Score: Not applicable
Calinski-Harabasz Score: Not applicable
Silhouette Score: Not applicable

Number of clusters: 1

DBSCAN-5_5_50 Clustering Evaluation:
Davies-Bouldin Score: 3.5703749108548695
Calinski-Harabasz Score: 59.47869727714488
Silhouette Score: -0.036010655650128405

Number of clusters: 23

DBSCAN-5_5_100 Clustering Evaluation:
Davies-Bouldin Score: 2.960831892148315
Calinski-Harabasz Score

In [24]:
def generate_clusters(message_embeddings,
                    n_neighbors,
                    n_components,
                    min_cluster_size,
                    random_state = None):
  """
  Generate HDBSCAN cluster object after reducing embedding dimensionality with UMAP
  """

  umap_embeddings = (umap.UMAP(n_neighbors=n_neighbors,
                              n_components=n_components,
                              metric='cosine',
                              random_state=random_state)
                          .fit_transform(message_embeddings))

  clusters = OPTICS(min_samples = min_cluster_size
                        ).fit_predict(umap_embeddings)

  return clusters

In [25]:
for n_neighbors in (5, 50, 100):
  for n_components in (3, 5, 20 , 50, len(features[0])):
    for min_cluster_size in (50, 100, 500, 1000):
      cluster_labels = generate_clusters(features,
          n_neighbors, n_components, min_cluster_size)
      evaluate_clustering(f"OPTICS-{n_neighbors}_{n_components}_{min_cluster_size}", features, cluster_labels)



OPTICS-5_3_50 Clustering Evaluation:
Davies-Bouldin Score: 3.5743032504671812
Calinski-Harabasz Score: 41.82720679647162
Silhouette Score: -0.06739666055073555

Number of clusters: 22

OPTICS-5_3_100 Clustering Evaluation:
Davies-Bouldin Score: 3.1294530117862873
Calinski-Harabasz Score: 144.2069831149283
Silhouette Score: -0.0017208491471406477

Number of clusters: 6

OPTICS-5_3_500 Clustering Evaluation:
Davies-Bouldin Score: 3.380835922663078
Calinski-Harabasz Score: 216.51570909849502
Silhouette Score: 0.06514385771886609

Number of clusters: 2

OPTICS-5_3_1000 Clustering Evaluation:
Davies-Bouldin Score: Not applicable
Calinski-Harabasz Score: Not applicable
Silhouette Score: Not applicable

Number of clusters: 1

OPTICS-5_5_50 Clustering Evaluation:
Davies-Bouldin Score: 3.4170748339964216
Calinski-Harabasz Score: 42.03598691974269
Silhouette Score: -0.07805200063188122

Number of clusters: 18

OPTICS-5_5_100 Clustering Evaluation:
Davies-Bouldin Score: 3.1671858809883378
Calinsk

In [26]:
def generate_clusters(message_embeddings,
                    n_neighbors,
                    n_components,
                    min_cluster_size,
                    random_state = None):
  """
  Generate HDBSCAN cluster object after reducing embedding dimensionality with UMAP
  """

  umap_embeddings = (umap.UMAP(n_neighbors=n_neighbors,
                              n_components=n_components,
                              metric='cosine',
                              random_state=random_state)
                          .fit_transform(message_embeddings))

  clusters = HDBSCAN(min_samples = min_cluster_size,
                              metric='euclidean').fit_predict(umap_embeddings)

  return clusters

In [27]:
for n_neighbors in (5, 50, 100):
  for n_components in (3, 5, 20 , 50, len(features[0])):
    for min_cluster_size in (50, 100, 500, 1000):
      cluster_labels = generate_clusters(features,
          n_neighbors, n_components, min_cluster_size)
      evaluate_clustering(f"HBDBSCAN-{n_neighbors}_{n_components}_{min_cluster_size}", features, cluster_labels)



HBDBSCAN-5_3_50 Clustering Evaluation:
Davies-Bouldin Score: 4.694847802874475
Calinski-Harabasz Score: 114.8684982524768
Silhouette Score: -0.012678710275221494

Number of clusters: 6

HBDBSCAN-5_3_100 Clustering Evaluation:
Davies-Bouldin Score: 3.569300376793256
Calinski-Harabasz Score: 132.53391343818626
Silhouette Score: 0.023431750257289666

Number of clusters: 3

HBDBSCAN-5_3_500 Clustering Evaluation:
Davies-Bouldin Score: 5.674620538966689
Calinski-Harabasz Score: 154.00275836578288
Silhouette Score: 0.04953140817325229

Number of clusters: 3

HBDBSCAN-5_3_1000 Clustering Evaluation:
Davies-Bouldin Score: Not applicable
Calinski-Harabasz Score: Not applicable
Silhouette Score: Not applicable

Number of clusters: 1

HBDBSCAN-5_5_50 Clustering Evaluation:
Davies-Bouldin Score: 4.622272293402341
Calinski-Harabasz Score: 131.71215721316767
Silhouette Score: 0.031285454584151

Number of clusters: 5

HBDBSCAN-5_5_100 Clustering Evaluation:
Davies-Bouldin Score: 4.309961223077736
Cal

In [28]:
print('Finished')

Finished
